# LLM-powered AI Agents

Table of contents
1. Understanding LLMs
2. Tools
3. Chat-based AI Agents
4. Service-based AI agents
5. Multi-Agents

In [1]:
import json
import pandas as pd
import random
from pathlib import Path
from datetime import datetime
from sklearn.metrics import accuracy_score
from pydantic import BaseModel, Field
from typing import Any
from io import StringIO
from language_models.agents.chain import AgentChain
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole
from language_models.tools.tool import Tool
from language_models.proxy_client import BTPProxyClient
from language_models.agents.react import ReActAgent
from language_models.tools.earthquake import earthquake_tools
from language_models.tools.current_date import current_date_tool
from language_models.settings import settings
from pprint import pprint

In [2]:
proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## 1. Understanding LLMs

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-35-turbo",
    max_tokens=256,
    temperature=0.0,
)

In [4]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!"""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Sentiment: Positive


In [5]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!

Respond with positive or negative."""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

positive


In [6]:
system_prompt = "Take the following movie review and determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Wow! This movie was incredible. The acting was superb, and the plot kept me on the edge of my seat. I highly recommend it!"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

1


In [7]:
system_prompt = "Take the following movie review determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
pprint(response)

("I'm sorry, I am an AI language model and I do not have access to real-time "
 'weather information. I recommend checking a reliable weather website or '
 'using a weather app to get the most accurate and up-to-date forecast for '
 'Seattle.')


In [8]:
system_prompt = """Take the following movie review and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a movie review, respond with -1."""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

-1


## 2. Tools

In [9]:
prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12" # answer: $1,289.98

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Total Raw Cost = $1,290.98


In [10]:
def calculator(expression: str) -> Any:
    return eval(expression)

class Calculator(BaseModel):
    expression: str = Field(description="A math expression.")

calculator_tool = Tool(
    func=calculator,
    name="Calculator",
    description="Use this tool when you want to do calculations.",
    args_schema=Calculator
)

print(calculator_tool)

tool name: Calculator, tool description: Use this tool when you want to do calculations., tool input: {{'expression': {{'description': 'A math expression.', 'title': 'Expression', 'type': 'string'}}}}


In [11]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Please ALWAYS use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use. Must be on of {calculator_tool.name}",
  "tool_input": "Valid keyword arguments",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "You should always think about what to do consider previous and subsequent steps",
    "tool": "Calculator",
    "tool_input": {
        "expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"
    }
}


In [12]:
print(calculator(**response["tool_input"]))

1289.98


In [13]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Please ALWAYS use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use. Must be on of {calculator_tool.name}",
  "tool_input": "Valid keyword arguments",
}}

Observation: tool result
... (this Thought/Tool/Tool Input/Observation can repeat N times)

When you know the answer, you MUST use the following JSON format:
{{
  "thought": "I now know what to respond",
  "tool": "Final Answer",
  "tool_input": "The final answer to the question",
}}"""

prompt = """Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12

This was your previous work:
Thought: The user wants me to calculate the total raw cost. I will use the Calculator tool.
Tool: Calculator
Tool Input: {"expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"}
Observation: Tool Response: 1289.98"""

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "I now know what to respond",
    "tool": "Final Answer",
    "tool_input": "The total raw cost is $1289.98"
}


## 3. Chat-based AI Agents

### Earthquake

In [14]:
system_prompt = "You are an United States Geological Survey expert who can answer questions regarding earthquakes."

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

earthquake_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{question}",
    task_prompt_variables=["question"],
    tools=earthquake_tools + [current_date_tool],
    output_format=Output,
    iterations=10,
)

In [15]:
response = earthquake_agent.invoke({"question": "How many earthquakes occurred today?"})

23/05/24 21:34:28 INFO Prompt:
How many earthquakes occurred today?
23/05/24 21:34:33 INFO Raw response:
{
  "thought": "I need to get the current date first to determine the start and end times for the earthquake query.",
  "tool": "Current Date",
  "tool_input": {}
}
23/05/24 21:34:33 INFO Thought:
I need to get the current date first to determine the start and end times for the earthquake query.
23/05/24 21:34:33 INFO Tool:
Current Date
23/05/24 21:34:33 INFO Tool input:
{}
23/05/24 21:34:33 INFO Tool response:
2024-05-23 21:34:33.569769
23/05/24 21:34:42 INFO Raw response:
{
  "thought": "Now that I have the current date, I can use it to query the number of earthquakes that occurred today.",
  "tool": "Count Earthquakes",
  "tool_input": {
    "start_time": "2024-05-23T00:00:00",
    "end_time": "2024-05-23T23:59:59"
  }
}
23/05/24 21:34:42 INFO Thought:
Now that I have the current date, I can use it to query the number of earthquakes that occurred today.
23/05/24 21:34:42 INFO Too

In [16]:
print(response.final_answer["content"])

There were 151 earthquakes today.


In [17]:
response = earthquake_agent.invoke({"question": "Show me 3 with a magnitude > 5."})

23/05/24 21:34:47 INFO Prompt:
Show me 3 with a magnitude > 5.
23/05/24 21:34:56 INFO Raw response:
{
  "thought": "I need to query earthquakes that occurred today with a magnitude greater than 5 and limit the results to 3.",
  "tool": "Query Earthquakes",
  "tool_input": {"start_time": "2024-05-23T00:00:00", "end_time": "2024-05-23T23:59:59", "min_magnitude": 5, "limit": 3}
}
23/05/24 21:34:56 INFO Thought:
I need to query earthquakes that occurred today with a magnitude greater than 5 and limit the results to 3.
23/05/24 21:34:56 INFO Tool:
Query Earthquakes
23/05/24 21:34:56 INFO Tool input:
{'start_time': '2024-05-23T00:00:00', 'end_time': '2024-05-23T23:59:59', 'min_magnitude': 5, 'limit': 3}
23/05/24 21:34:56 INFO Tool response:
{'type': 'FeatureCollection', 'metadata': {'generated': 1716492896000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-05-23T00%3A00%3A00&endtime=2024-05-23T23%3A59%3A59&limit=3&mindepth=-100&maxdepth=1000&minmagnitu

In [18]:
print(response.final_answer["content"])

Here are 3 earthquakes that occurred today with a magnitude greater than 5:

1. A magnitude 5 earthquake occurred 68 km SW of Gerāsh, Iran. [Details](https://earthquake.usgs.gov/earthquakes/eventpage/us6000n0iw)

2. A magnitude 5.6 earthquake occurred at the Southwest Indian Ridge. [Details](https://earthquake.usgs.gov/earthquakes/eventpage/us6000n0ee)

3. A magnitude 5.5 earthquake occurred at the Carlsberg Ridge. [Details](https://earthquake.usgs.gov/earthquakes/eventpage/us6000n0e7)


In [19]:
response = earthquake_agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

23/05/24 21:35:13 INFO Prompt:
Can MegaQuakes really happen? Like a magnitude 10 or larger?
23/05/24 21:36:27 INFO Raw response:
{'thought': 'A magnitude 10 or larger earthquake, often referred to as a "megaquake", is theoretically possible but extremely unlikely. The magnitude of an earthquake is related to the length of the fault on which it occurs - the longer the fault, the larger the earthquake. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960, and it occurred on a fault that was over 1,000 miles long and 150 miles wide. To get a magnitude 10 earthquake would require a fault three times the length of the Chile fault, and there are no known faults of this length on Earth. Therefore, while a magnitude 10 earthquake is theoretically possible, it is extremely unlikely to occur.', 'tool': 'Final Answer', 'tool_input': {'content': 'A magnitude 10 or larger earthquake, often referred to as a "megaquake", is theoretically possible but extremely unlikely. The magni

In [20]:
pprint(response.final_answer["content"])

("While it's theoretically possible for a magnitude 10 earthquake to occur, "
 "it's extremely unlikely. The largest earthquake ever recorded was a "
 'magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake would require a '
 'fault line capable of slipping along a distance of about 1000 kilometers, '
 "which doesn't exist on Earth. So, while it's theoretically possible, it's "
 "not practically feasible given the Earth's current geological makeup.")


## 4. Service-based AI Agents

### Contract Drafting

In [21]:
system_prompt = """You are a corporate lawyer. Take the follow bullet points and generate a draft of a section for a contract. Make it lengthy."""

task_prompt = """Bullet points:
{section}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class ContractSection(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")

contract_drafting_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["section", "section_name"],
    tools=None,
    output_format=ContractSection,
    iterations=3,
)

In [22]:
def generate_contract(contract_sections: list[str]) -> str:
    sections = []
    for contract_section in contract_sections:
        response = contract_drafting_agent.invoke({"section": contract_section})
        section = str(response.final_answer["title"]) + "\n\n" + str(response.final_answer["content"])
        sections.append(section)
        contract_drafting_agent.reset()
    return "\n\n".join(sections)

In [23]:
definitions = "Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA."

amendment = """INVOICING AND PAYMENT TERMS
Clause 12.1(ii) of the GMA shall be cancelled and substituted as follow:
[*****]
[*****]
[*****]

Any other provision of Clause 12 shall remain in full force and effect.

PRICE CONDITIONS
(i) Clause 3.2 of the Exhibit 14 of the GMA shall be cancelled and substituted as follow:
“3.2 Technical conditions for prices adjustment
The prices set out in this Exhibit 14 shall be modified every [*****] at the occasion of the invoicing reconciliation pursuant to Clause 11
(“Reconciliation”) if the Standard Operations of the Aircraft, analyzed at the time of the adjustment (all calculations are made with figures corresponding to [*****], change by more or less
[*****] with respect to the estimated values of the same parameters, considered at the time of commencement of the Term.
As from the date this Agreement enters into force, the Parties agree to take into account the following basic operating parameters (the
“Standard Operations”) as a reference for the above calculation:
[*****]
[*****]
[*****]"""

effective_date_and_duration = "Amendment is effective starting on the date of its signature by both Parties."

confidentiality = """Confidential Information released by either of the Parties (the “Disclosing Party”) to the
other Party (the “Receiving Party”) shall not be released in whole or in part to any third party:
- Not to deliver, disclose or publish it to any third party including subsidiary companies and companies having an interest in its capital
- Use Confidential Information solely for the purpose of this Amendment
- Disclose the Confidential Information only to those of its direct employees
- Not to duplicate the Confidential Information nor to copy

Any Confidential Information shall remain the property of the Disclosing Party.

The Receiving Party hereby acknowledges and recognises that Confidential Information is protected by copyright Laws and related
international treaty provisions, as the case may be.

This shall survive termination or expiry of this Amendment for a period of five (5) years following such End Date."""

governing_law = """Pursuant to and in accordance with Section 5-1401 of the New York General Obligations Law.

Arbitration: in the event of a dispute arising out of or relating to this Amendment, including without limitation disputes regarding the
existence, validity or termination of this Amendment (a “Dispute”), either Party may notify such Dispute to the other through service of a
written notice (the “Notice of Dispute”).

Arbitration, and any proceedings, and meetings incidental to or related to the arbitration process, shall take place in New York.

Arbitration shall be kept confidential and the existence of the proceeding and any element.

During any period of negotiation or arbitration, the Parties shall continue to meet their respective obligations.

Notwithstanding any provision of this the Parties may, at any time, seek and decide to settle a Dispute.

Judgment upon any award may be entered in any court having jurisdiction.

Recourse to jurisdictions is expressly excluded except as provided for in the ICC Rules of Conciliation and Arbitration."""

miscellaneous = """Amendment contains the entire agreement between the Parties regarding the subject-matter.

Amendment shall not be varied or modified except by a written document duly signed."""

contract = generate_contract(
    contract_sections=[
        definitions,
        amendment,
        effective_date_and_duration,
        confidentiality,
        governing_law,
        miscellaneous,
    ]
)

23/05/24 21:36:38 INFO Prompt:
Bullet points:
Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA.
23/05/24 21:37:06 INFO Raw response:
{
  "thought": "The bullet point provided is about the interpretation of capitalized terms in the amendment and their relation to the GMA. I will draft a section that elaborates on this point, ensuring that it is clear and comprehensive.",
  "tool": "Final Answer",
  "tool_input": {
    "title": "Interpretation of Capitalised Terms",
    "content": "This section of the Agreement, titled 'Interpretation of Capitalised Terms', is intended to provide clarity on the usage and interpretation of capitalised terms within this Amendment. Any term that is capitalised, whether it is in singular or plural form, shall be understood to have the same meaning as defined in the General Master Agreement (GMA). This applies to all instances where such capitalised terms are used within this Amendment. The purpose of this 

In [24]:
pprint(contract)

('Interpretation of Capitalised Terms\n'
 '\n'
 "This section of the Agreement, titled 'Interpretation of Capitalised Terms', "
 'is intended to provide clarity on the usage and interpretation of '
 'capitalised terms within this Amendment. Any term that is capitalised, '
 'whether it is in singular or plural form, shall be understood to have the '
 'same meaning as defined in the General Master Agreement (GMA). This applies '
 'to all instances where such capitalised terms are used within this '
 'Amendment. The purpose of this provision is to ensure consistency and '
 'coherence in the interpretation of the terms across different documents '
 'related to this Agreement. It is important to note that this provision '
 'applies irrespective of the number (singular or plural) in which the term is '
 'used. The interpretation should always align with the definition provided in '
 'the GMA. This provision is fundamental to the understanding and '
 'interpretation of this Amendment and shou

### Sentiment Analysis

In [25]:
df_tweets = pd.read_csv("./data/tweets.csv.gz", compression="gzip", encoding="latin-1", names=["sentiment", "id", "date", "query", "user", "tweet"])
df_tweets = df_tweets.dropna()
df_tweets = df_tweets.where(df_tweets.sentiment != 2)
df_tweets["sentiment"] = df_tweets["sentiment"].map({4: 1, 0: 0})
df_tweets_sampled = df_tweets.sample(n=10)
df_tweets_sampled.head()

,sentiment,id,date,query,user,tweet
705201,0,2256287147,Sat Jun 20 12:32:46 PDT 2009,NO_QUERY,ClauBand,@ThiOliveiras To no ar Thi!!!
1060701,1,1963550470,Fri May 29 12:53:37 PDT 2009,NO_QUERY,belolats,the jeepney trip was surprisingly calming. may...
736088,0,2264932638,Sun Jun 21 05:20:50 PDT 2009,NO_QUERY,PoynterJones,"@miss_pipedream It's hard though Like, my fav..."
651111,0,2237783457,Fri Jun 19 06:21:16 PDT 2009,NO_QUERY,ashley_katty,I hate humidity!
1001211,1,1880081106,Fri May 22 00:02:27 PDT 2009,NO_QUERY,akr93,4ever! Haha love this song


In [26]:
system_prompt = """Take the following tweet and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a tweet, respond with -1."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class Sentiment(BaseModel):
    sentiment: int = Field(description="The sentiment of the tweet.")

sentiment_analysis_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Tweet:\n{tweet}",
    task_prompt_variables=["tweet"],
    tools=None,
    output_format=Sentiment,
    iterations=3,
)

In [27]:
def classify_sentiment(tweet: str) -> int:
    response = sentiment_analysis_agent.invoke({'tweet': tweet})
    sentiment_analysis_agent.reset()
    return response.final_answer['sentiment'] or 0

In [28]:
df_tweets_sampled["prediction"] = [classify_sentiment(tweet) for tweet in df_tweets_sampled.tweet]

23/05/24 21:39:28 INFO Prompt:
Tweet:
@ThiOliveiras To no ar Thi!!! 
23/05/24 21:39:34 INFO Raw response:
{
  "thought": "The tweet doesn't contain any sentiment, it's just a statement.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": -1}
}
23/05/24 21:39:34 INFO Thought:
The tweet doesn't contain any sentiment, it's just a statement.
23/05/24 21:39:34 INFO Final answer:
{'sentiment': -1}
23/05/24 21:39:34 INFO Prompt:
Tweet:
the jeepney trip was surprisingly calming. maybe because of the drizzle and the faceless passengers. liked the driver's music mix too. 
23/05/24 21:39:42 INFO Raw response:
{
  "thought": "The tweet seems to express a positive sentiment. The user found the jeepney trip calming, enjoyed the drizzle, the anonymity of the passengers, and the driver's music mix.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": 1}
}
23/05/24 21:39:42 INFO Thought:
The tweet seems to express a positive sentiment. The user found the jeepney trip calming, enjoyed the dri

In [29]:
print(f"Accuracy: {accuracy_score(df_tweets_sampled.sentiment, df_tweets_sampled.prediction)}")

Accuracy: 0.6


### Structuring Unstructured Data

In [30]:
path = Path("./data/jobs")
filenames = [file.name for file in path.iterdir() if file.is_file()]
filenames = random.sample(filenames, 5)

jobs = []
for filename in filenames:
    file_path = path / filename
    with open(file_path, "r", encoding="utf-8", errors="replace") as file:
        content = file.read()
        jobs.append(content)

In [31]:
system_prompt = "Take the following job and extract data about the job."

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    float=0.0,
)

class Job(BaseModel):
    job_title: str = Field(description="The job title.")
    job_class_no: int = Field(description="The job class code.")
    job_duties: str = Field(description="The duties of the job.")
    open_date: str = Field(description="When the position was opened. Format: DD-MM-YYYY.")
    salary: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")
    deadline: str = Field(description="The application deadline. Format: DD-MM-YYYY")
    application_form: str = Field(description="The form of the application (e.g. online, fax, email).")
    where_to_apply: str = Field(description="The url to apply at or location to send the fax or email address.")

job_data_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job}",
    task_prompt_variables=["job"],
    tools=None,
    output_format=Job,
    iterations=3,
)

In [32]:
def extract_jobs(jobs: list[str]) -> pd.DataFrame:
    data = []
    for job in jobs:
        response = job_data_agent.invoke({"job": job})
        data.append(response.final_answer)
        job_data_agent.reset()
    return pd.DataFrame(data)

In [33]:
df_jobs = extract_jobs(jobs)

23/05/24 21:40:30 INFO Prompt:
Job description:
CUSTOMER SERVICE REPRESENTATIVE
	
Class Code:       1230
Open Date:  02-09-18
(Exam Open to All, including Current City Employees)

ANNUAL SALARY

$57,148 to $71,012; $60,405 to $75,042; and $65,980 to $81,954

NOTES:

1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.
2. Annual salary is at the beginning of the pay range. The current salary range is subject to change. Please confirm the stating salary with the hiring department before accepting a job offer. 
3. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.

DUTIES

A Customer Service Representative has direct public contact including in-person transactions; use of the telephone and Internet to receive customer requests for service; provide information and respond in writing to customers of the Department of Water and Power r

In [34]:
df_jobs.head()

,job_title,job_class_no,job_duties,open_date,salary,deadline,application_form,where_to_apply
0,CUSTOMER SERVICE REPRESENTATIVE,1230,A Customer Service Representative has direct p...,02-09-18,"[$57,148 to $71,012, $60,405 to $75,042, $65,9...","11:59 PM, THURSDAY, FEBRUARY 22, 2018",ON-LINE,https://www.governmentjobs.com/careers/lacity
1,INVESTMENT OFFICER,9146,An Investment Officer may be responsible for o...,06-15-18,"[$93,354 to $136,471, $116,301 to $170,025, $1...","THURSDAY, JUNE 28, 2018",online,https://www.governmentjobs.com/careers/lacity
2,UTILITY EXECUTIVE SECRETARY,1336,A Utility Executive Secretary performs highly ...,04-28-17,"[$75,188 to $79,385, $78,383 to $82,747, $81,9...","THURSDAY, MAY 11, 2017",online,https://www.governmentjobs.com/careers/lacity/...
3,CEMENT FINISHER,3353,"A Cement Finisher does skilled work in mixing,...",03-09-18,"[$79,244 (flat-rated), The salary range in the...","THURSDAY, MARCH 22, 2018",Online,https://www.governmentjobs.com/careers/lacity
4,AIRPORT INFORMATION SPECIALIST,1783,An Airport Information Specialist provides and...,12-11-15,"[$42,991 to $62,869, $53,724 to $78,529]",24-12-2015,Online,http://agency.governmentjobs.com/lacity/defaul...


## 5. Multi-Agents

### Comparing Unstructured Data

In [35]:
def get_job(path: str) -> str:
    with open(path, "r", encoding="utf-8") as file:
        content = file.read()
        return content

job1 = get_job("./data/jobs/ELECTRICAL ENGINEERING ASSOCIATE 7525 093016 REV 100416.txt")
job2 = get_job("./data/jobs/ELECTRICAL MECHANIC 3841 012017.txt")

In [36]:
system_prompt = "Take the following job and extract data about the job"

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    float=0.0,
)

class Job1(BaseModel):
    job1_title: str = Field(description="The job title.")
    job1_duties: str = Field(description="The duties of the job.")
    salary1: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

class Job2(BaseModel):
    job2_title: str = Field(description="The job title.")
    job2_duties: str = Field(description="The duties of the job.")
    salary2: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

job_agent1 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job1}",
    task_prompt_variables=["job1"],
    tools=None,
    output_format=Job1,
    iterations=10,
)

job_agent2 = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job2}",
    task_prompt_variables=["job2"],
    tools=None,
    output_format=Job2,
    iterations=10,
)

In [37]:
system_prompt = "Take the following 2 job descriptions and respond with the similarities and differences of the jobs."

task_prompt = """Compare the 2 given job descriptions:

Job 1:
Job title: {job1_title}
Job duties:
{job1_duties}
Salary:
{salary1}


Job 2:
Job title: {job2_title}
Job duties:
{job2_duties}
Salary:
{salary2}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    float=0.0,
)

class JobComparison(BaseModel):
    similarities: str = Field(description="The job similarities.")
    differences: str = Field(description="The job differences.")

job_comparison_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["job1_title", "job1_duties", "salary1", "job2_title", "job2_duties", "salary2"],
    tools=None,
    output_format=JobComparison,
    iterations=10,
)

In [38]:
chain = AgentChain(
    chain=[job_agent1, job_agent2, job_comparison_agent],
    chain_variables=["job1", "job2"],
)

In [39]:
response = chain.invoke({"job1": job1, "job2": job2})

23/05/24 21:43:17 INFO Prompt:
Job description:
ELECTRICAL ENGINEERING ASSOCIATE
Class Code:       7525
Open Date:  09-30-16
REVISED: 10-04-16
 (Exam Open to All, including Current City Employees)
ANNUAL SALARY 

$66,231 to $94,252; $74,082 to $105,444; $82,497 to $117,346; and $89,638 to $127,556
The salary in the Department of Water and Power is $77,360 to $96,110; $91,934 to $114,213; $99,722 to $123,881; and $107,156 to 
$133,130

NOTES:

1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.
2. For information regarding reciprocity between City of Los Angeles departments and LADWP, go to: http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.
3. The current salary range is subject to change. You may confirm the starting salary with the hiring department before accepting a job offer.

DUTIES

An Electrical Engineering Associate performs professional electrical engineering work in the preparation of designs, plans, specifications

In [40]:
pprint(response.final_answer["similarities"])

('Both jobs are in the electrical field and involve working with electrical '
 'systems and equipment. They both may involve work in various facilities and '
 'buildings.')


In [41]:
pprint(response.final_answer["differences"])

('The Electrical Engineering Associate is more focused on design, planning, '
 'and quality assurance, while the Electrical Mechanic is more hands-on, '
 'dealing with the installation and maintenance of electrical circuits and '
 'equipment. The Electrical Engineering Associate may also be involved in code '
 'enforcement functions. The Electrical Mechanic may work at heights, near '
 'hazardous materials, and in confined spaces. The salary ranges also differ, '
 'with the Electrical Engineering Associate having a wider range and '
 'potentially higher salary.')


### Machine Learning Code Generation

In [42]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

Respond with 1 of the following machine learning problems:
- Classification
- Regression
- Clustering
- Time series forecasting"""

task_prompt = """Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
{problem_description}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class ModelingProblem(BaseModel):
    modeling_problem: str = Field(description="The machine learning problem.")

problem_finder_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["problem_description", "dataset_size", "dataset_schema"],
    tools=None,
    output_format=ModelingProblem,
    iterations=5,
)

In [43]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

You can solve machine learning problems for:
- Classification
- Regression
- Clustering
- Time series forecasting

You have access to the following Python libraries:
- pandas
- numpy
- scikit-learn"""

task_prompt = """Given the following machine learning problem, respond with Python code.

Modeling problem: {modeling_problem}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}
First 10 rows of dataset:
{dataset_snippet}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=512,
    float=0.0,
)

class AutoMLCode(BaseModel):
    code: str = Field(description="The Python machine learning code.")

ml_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["modeling_problem", "dataset_size", "dataset_schema", "dataset_snippet"],
    tools=None,
    output_format=AutoMLCode,
    iterations=10,
)

In [44]:
ml_chain = AgentChain(
    chain=[problem_finder_agent, ml_agent],
    chain_variables=["problem_description", "dataset_size", "dataset_schema", "dataset_snippet"]
)

In [45]:
info_str = StringIO()
df_tweets.info(buf=info_str)
dataset_schema = info_str.getvalue()

In [46]:
response = ml_chain.invoke({
    "problem_description": "I want to classify the sentiment of tweets.",
    "dataset_size": len(df_tweets),
    "dataset_schema": dataset_schema,
    "dataset_snippet": str(df_tweets.head(10).to_markdown())
})

23/05/24 21:45:08 INFO Prompt:
Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
I want to classify the sentiment of tweets.

Dataset:
Number of rows: 1600000
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

23/05/24 21:45:14 INFO Raw response:
{
  "thought": "Given the problem description and the dataset, it seems like the user wants to predict the sentiment of tweets, which is a categorical variable. This is a typical example of a classification problem in machine learning.",
  "tool

In [47]:
print(response.final_answer["code"])

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Assuming that `df` is your DataFrame
X = df['tweet']
y = df['sentiment']

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the training data and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = vectorizer.transform(X_test)

# Create a Naive Bayes classifier
clf = MultinomialNB()

# Fit the classifier to the training data
clf.fit(X_train_tfidf, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test_tfidf)

# Print a classification report
print(classification_report(y_test, y_pred))


### Forecasting

In [48]:
from datetime import timedelta
from language_models.tools.forecasting import get_earthquakes_data, ml_model

In [49]:
class Forecast(BaseModel):
    start_time: str = Field(None, description='Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')
    end_time: str = Field(None, description='Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')

def forecast(start_time = None, end_time = None):
    if start_time is None:
        start_time = (datetime.now() - timedelta(days=30)).date()
    if end_time is None:
        end_time = (datetime.now().date())
    df = get_earthquakes_data('https://earthquake.usgs.gov/fdsnws/event/1/query?', start_time, end_time)
    df_pred = ml_model.predict(df)
    return {'predictions': df_pred.to_dict(orient='records')}

In [50]:
forecasting_tool = Tool(func=forecast, name='Forecast', description='Test forecast model on real-time events.', args_schema=Forecast)

In [51]:
task_prompt = """{question}

Use the Current Date tool and respond with the start time and end time."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=256,
    float=0.0,
)

class DateRange(BaseModel):
    start_time: str = Field(description="The start date. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")
    end_time: str = Field(description="The end date. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")

time_wizard_agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt=task_prompt,
    task_prompt_variables=["question"],
    tools=[current_date_tool],
    output_format=DateRange,
    iterations=5,
)

In [52]:
forecast_chain = AgentChain(
    chain=[time_wizard_agent, forecasting_tool],
    chain_variables=["question"],
)

In [53]:
response = forecast_chain.invoke({"question": "Run a forecast using the past week as data."})

23/05/24 21:47:08 INFO Prompt:
Run a forecast using the past week as data.

Use the Current Date tool and respond with the start time and end time.
23/05/24 21:47:17 INFO Raw response:
{
  "thought": "First, I need to get the current date to calculate the start and end times for the past week.",
  "tool": "Current Date",
  "tool_input": {}
}
23/05/24 21:47:17 INFO Thought:
First, I need to get the current date to calculate the start and end times for the past week.
23/05/24 21:47:17 INFO Tool:
Current Date
23/05/24 21:47:17 INFO Tool input:
{}
23/05/24 21:47:17 INFO Tool response:
2024-05-23 21:47:17.162202
23/05/24 21:47:45 INFO Raw response:
{
  "thought": "Now that I have the current date, I can calculate the start and end times for the past week. The end time would be the current date and the start time would be the current date minus 7 days.",
  "tool": "Final Answer",
  "tool_input": {
    "start_time": "2024-05-16T21:47:17.162202",
    "end_time": "2024-05-23T21:47:17.162202"
  

In [54]:
print(response.final_answer["Forecast"])

{'predictions': [{'time': Timestamp('2024-05-16 21:48:15.980000+0000', tz='UTC'), 'prediction': 2.114732, 'latitude': 38.7973327636719, 'longitude': -122.766502380371, 'mag': 0.7, 'id': 'nc75007856', 'place': '2 km  of The Geysers, CA', 'location': '2 km  of The Geysers, CA'}, {'time': Timestamp('2024-05-16 22:11:44.500000+0000', tz='UTC'), 'prediction': 2.14891, 'latitude': 35.949, 'longitude': -117.6805, 'mag': 0.9, 'id': 'ci40587719', 'place': '20 km E of Little Lake, CA', 'location': '20 km E of Little Lake, CA'}, {'time': Timestamp('2024-05-16 22:21:30.359000+0000', tz='UTC'), 'prediction': 2.564279, 'latitude': 34.99916667, 'longitude': -97.67616667, 'mag': 1.3, 'id': 'ok2024jqer', 'place': '5 km SW of Dibble, Oklahoma', 'location': '5 km SW of Dibble, Oklahoma'}, {'time': Timestamp('2024-05-16 22:24:38.106000+0000', tz='UTC'), 'prediction': 2.174436, 'latitude': 61.6649, 'longitude': -146.8134, 'mag': 0.9, 'id': 'ak0246axv8s3', 'place': '35 km SSE of Eureka Roadhouse, Alaska', '

### LLM-backed Tools

In [55]:
earthquake_agent.reset()
problem_finder_agent.reset()
ml_agent.reset()

In [56]:
class EarthquakeAgent(BaseModel):
    question: str = Field(description="The question regarding earthquakes.")

def answer_earthquake_questions(question: str) -> Any:
    response = earthquake_agent.invoke({"question": question})
    return response.final_answer

earthquake_agent_tool = Tool(
    func=answer_earthquake_questions,
    name="Earthquake Agent",
    description="Use this tool to answer questions about earthquakes.",
    args_schema=EarthquakeAgent,
)

class MLAgent(BaseModel):
    problem_description: str = Field(description="The user problem.")
    dataset_size: int = Field(description="The size of the dataset."),
    dataset_schema: str = Field(description="The dataset schema or information."),
    dataset_snippet: str = Field(description="The dataset snippet aka the first couple of rows of the dataset.")

def generate_ml_code(problem_description: str, dataset_size: int, dataset_schema: str, dataset_snippet: str) -> Any:
    response = ml_chain.invoke({
        "problem_description": problem_description,
        "dataset_size": dataset_size,
        "dataset_schema": dataset_schema,
        "dataset_snippet": dataset_snippet,
    })
    return response.final_answer

ml_agent_tool = Tool(
    func=generate_ml_code,
    name="ML Agent",
    description="Use this tool to generate machine learning code given a problem.",
    args_schema=MLAgent,
)

In [57]:
system_prompt = """You are an Agent that delegates tasks to other Agents by using the appropriate tools.

Use the Earthquake Agent when the question is about earthquakes.

Use the ML Agent when the user wants you to generate machine learning code."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=2048,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

almighty_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{prompt}",
    task_prompt_variables=["prompt"],
    tools=[earthquake_agent_tool, ml_agent_tool],
    output_format=Output,
    iterations=10,
)

/opt/homebrew/Caskroom/miniforge/base/envs/llm-powered-ai-agents/lib/python3.11/site-packages/pydantic/json_schema.py:2158: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The size of the dataset.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/llm-powered-ai-agents/lib/python3.11/site-packages/pydantic/json_schema.py:2158: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The dataset schema or information.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [58]:
response = almighty_agent.invoke({"prompt": "How many earthquakes happened today?"})

23/05/24 21:47:52 INFO Prompt:
How many earthquakes happened today?
23/05/24 21:47:56 INFO Raw response:
{
  "thought": "The user is asking about earthquakes. I should use the Earthquake Agent to get this information.",
  "tool": "Earthquake Agent",
  "tool_input": {"question": "How many earthquakes happened today?"}
}
23/05/24 21:47:56 INFO Thought:
The user is asking about earthquakes. I should use the Earthquake Agent to get this information.
23/05/24 21:47:56 INFO Tool:
Earthquake Agent
23/05/24 21:47:56 INFO Tool input:
{'question': 'How many earthquakes happened today?'}
23/05/24 21:47:56 INFO Prompt:
How many earthquakes happened today?
23/05/24 21:48:00 INFO Raw response:
{
  "thought": "I need to get the current date first to determine the start and end times for the earthquake query.",
  "tool": "Current Date",
  "tool_input": {}
}
23/05/24 21:48:00 INFO Thought:
I need to get the current date first to determine the start and end times for the earthquake query.
23/05/24 21:48

In [59]:
print(response.final_answer["content"])

There were 155 earthquakes today.


In [60]:
prompt = f"""Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: {len(df_tweets)}
Schema:
{dataset_schema}
First 10 rows of dataset:
{df_tweets.head(10).to_markdown()}"""

response = almighty_agent.invoke({"prompt": prompt})

23/05/24 21:48:17 INFO Prompt:
Give me code to train a model that predicts the sentiment of tweet.

Dataset:
Number of rows: 1600000
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

First 10 rows of dataset:
|    |   sentiment |         id | date                         | query    | user            | tweet                                                                                                               |
|---:|------------:|-----------:|:-----------------------------|:---------|:----------------|:-----------------------------------

In [61]:
print(response.final_answer["content"])

Here is the Python code to train a model that predicts the sentiment of a tweet:

```python
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

# Assuming df is the DataFrame

# Preprocessing: Remove unnecessary columns

df = df.drop(columns=['id', 'date', 'query', 'user'])

# Split the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['sentiment'], test_size=0.2, random_state=42)

# Create a pipeline that transforms the data and applies the model

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

# Train the model

pipeline.fit(X_train, y_train)

# Predict the test set results

y_pred = pipeline.predict(X_test)

# Print the classification report

p